In [5]:
using NBInclude
@nbinclude("Eulergrid.ipynb")
@nbinclude("Operators.ipynb")
abstract type Abstractboundary end
abstract type Abstractboundaryfunction end
abstract type oneDdelta end


In [6]:
mutable struct PeriodicLagrangianBoundary

    N::Int64 
    ## Location of bnd and forces.
    bnd_loc::Matrix{Float64}
	stretch_force::Matrix{Float64}

    ##Constants for force
    stretch_kp::Vector{Float64}
	stretch_km::Vector{Float64}
	gammap::Vector{Float64}
	gammam::Vector{Float64}
    
    ## Arrays from approximation of tangent vectors.
    taup::Matrix{Float64}
	taum::Matrix{Float64}
	tau0::Matrix{Float64}
	norm0::Matrix{Float64}

    ##Arrays from approximation of arc length
    dsp::Vector{Float64}
	dsm::Vector{Float64}
	ds0::Vector{Float64}
end

function PeriodicLagrangianBoundary(X::Vector{T},Y::Vector{T}) where T <: Real
    if ~(size(X)==size(Y))
        println("Size is not same")
    end
    N = length(X);
	bnd_loc = hcat(X,Y);
	taup = zeros(size(bnd_loc));
	taum = zeros(size(bnd_loc));
	norm0 = zeros(size(bnd_loc));
    stretch_force=0*bnd_loc
   
	kp = ones(N);
	
	km = ones(N);
	gammap = ones(N);
	gammam = ones(N);

	taup[1:N-1,:] = bnd_loc[2:N,:] - bnd_loc[1:N-1,:];
	taup[N,:] = bnd_loc[1,:] - bnd_loc[N,:];

	taum[2:N,:] = bnd_loc[2:N,:] - bnd_loc[1:N-1,:];
	taum[1,:] = bnd_loc[1,:] - bnd_loc[N,:];
	
	tau0 = (taup .+ taum)./2.0;

    
	dsp = @. sqrt(taup[:,1]^2 + taup[:,2]^2);
	dsm = @. sqrt(taum[:,1]^2 + taum[:,2]^2);
	ds0 = @. (dsp + dsm)/2;
    stretch_force[1,1]=kp[1]*(X[2]-2*X[1]+X[N])/(ds0[1])^2;   #second order definition for second derivative
    stretch_force[N,1]=kp[N]*(X[1]-2*X[N]+X[N-1])/(ds0[N])^2;
    for i in 2:N-1
        stretch_force[i,1]=kp[i]*(X[i+1]-2*X[i]+X[i-1])/(ds0[i])^2
    end
    stretch_force[1,2]=kp[1]*(Y[2]-2*Y[1]+Y[N])/(ds0[1])^2
    stretch_force[N,2]=kp[N]*(Y[1]-2*Y[N]+Y[N-1])/(ds0[N])^2;
    for i in 2:N-1
        stretch_force[i,2]=kp[i]*(Y[i+1]-2*Y[i]+Y[i-1])/(ds0[i])^2
    end
    
	norm0[:,1] = -tau0[:,2];
	norm0[:,2] = tau0[:,1];

	bnd::PeriodicLagrangianBoundary = PeriodicLagrangianBoundary(N,bnd_loc,stretch_force,kp,km,gammap,gammam,taup,taum,tau0,norm0,dsp,dsm,ds0);
	return bnd

end
    #This is a struct which holds data defined on the boundary. First scalar
mutable struct ScalarBndData <: Abstractboundaryfunction
	U::Vector{Float64}
	bnd::Abstractboundary

	function ScalarBndData(u::Vector{Float64},bnd::PeriodicLagrangianBoundary)
		if ~(length(u) == bnd.N)
			println("Size is not same")
		else
			return new(u,mybnd)
		end
	end
end

#And now vector
mutable struct VectorBndData <: Abstractboundaryfunction
	U::Vector{Float64}
	V::Vector{Float64}
	bnd::Abstractboundary

	function VectorBndData(u::Vector{Float64},v::Vector{Float64},bnd::PeriodicLagrangianBoundary)
		if ~(length(u) == bnd.N)
			println("Size is not same for first input");
		elseif ~(length(v) == mybnd.N)
			println("Size is not same for second input")
		else
			return new(u,v,mybnd)
		end
	end
end

### Spread and interpolation operators

function ScalarBndSpread(data::Vector{Float64},bnd::PeriodicLagrangianBoundary,grid::PeriodicEulergrid)
	if ~(length(data) == bnd.N)
		println("Size is not same.")
	end

	M = bnd.N;
	Nx = grid.Nx;
	Ny = grid.Ny;
    taup=bnd.taup
	output = zeros(Nx,Ny);

	for i = 1:M
		left = Int(floor(bnd.bnd_loc[i,1]/grid.dx + 0.5));
		xindeces = left-1:left+2;
		modxin = mod1.(xindeces,Nx);
		coords = grid.dx*(xindeces .- 0.5);
		distance = coords .- bnd.bnd_loc[i,1];

		xweights = PeskinDelta(distance/grid.dx)./grid.dx;


		bottom = Int(floor(bnd.bnd_loc[i,2]/grid.dy + 0.5));
		yindeces = bottom-1:bottom+2;
		modyin = mod1.(yindeces,Ny);
		coords = grid.dy*(yindeces .- 0.5);
		distance = coords .- bnd.bnd_loc[i,2];

		yweights = PeskinDelta(distance/grid.dy)./grid.dy;

		for (j,indx) in pairs(modxin)
			for (k,indy) in pairs(modyin)
				output[indx,indy] = output[indx,indy] + data[i]*xweights[j]*yweights[k]*bnd.ds0[i]
			end
		end

	end
	return output
end
function VectorBndSpread(data::Matrix{Float64},bnd::PeriodicLagrangianBoundary,grid::PeriodicEulergrid)
	if ~(size(data) == (N,2))
		println("Size is not same.")
	end

	M = bnd.N;
	Nx = grid.Nx;
	Ny = grid.Ny;
    taup=bnd.taup
	outputx = zeros(Nx,Ny);
    outputy=zeros(Nx,Ny);

	for i = 1:M
		left = Int(floor(bnd.bnd_loc[i,1]/grid.dx + 0.5));
		xindeces = left-1:left+2;
		modxin = mod1.(xindeces,Nx);
		coords = grid.dx*(xindeces .- 0.5);
		distance = coords .- bnd.bnd_loc[i,1];

		xweights = PeskinDelta(distance/grid.dx)./grid.dx;


		bottom = Int(floor(bnd.bnd_loc[i,2]/grid.dy + 0.5));
		yindeces = bottom-1:bottom+2;
		modyin = mod1.(yindeces,Ny);
		coords = grid.dy*(yindeces .- 0.5);
		distance = coords .- bnd.bnd_loc[i,2];

		yweights = PeskinDelta(distance/grid.dy)./grid.dy;

		for (j,indx) in pairs(modxin)
			for (k,indy) in pairs(modyin)
				outputx[indx,indy] = outputx[indx,indy] + (data[i,1])*xweights[j]*yweights[k]*bnd.ds0[i]
			end
		end
        for (j,indx) in pairs(modxin)
			for (k,indy) in pairs(modyin)
				outputy[indx,indy] = outputy[indx,indy] + (data[i,2])*xweights[j]*yweights[k]*bnd.ds0[i]
			end
		end

	end
	return outputx,outputy
end

function ScalarBndInterp(data::Matrix{Float64},bnd::PeriodicLagrangianBoundary,grid::PeriodicEulergrid)
	if ~(size(data) == (grid.Nx,grid.Ny))
		println("Size is not same.")
	end

	M = bnd.N;
	Nx = grid.Nx;
	Ny = grid.Ny;

	output = zeros(M);

	for i = 1:M
		left = Int(floor(mybnd.bnd_loc[i,1]/mygrid.dx + 0.5));
		xindeces = left-1:left+2;
		modxin = mod1.(xindeces,Nx);
		coords = grid.dx*(xindeces .- 0.5);
		distance = coords .- bnd.bnd_loc[i,1];

		xweights = PeskinDelta(distance/grid.dx)./grid.dx;


		bottom = Int(floor(bnd.bnd_loc[i,2]/grid.dy + 0.5));
		yindeces = bottom-1:bottom+2;
		modyin = mod1.(yindeces,Ny);
		coords = grid.dy*(yindeces .- 0.5);
		distance = coords .- bnd.bnd_loc[i,2];

		yweights = PeskinDelta(distance/grid.dy)./grid.dy;

		for (j,indx) in pairs(modxin)
			for (k,indy) in pairs(modyin)
				output[i] = output[i] + data[indx,indy]*xweights[j]*yweights[k]*grid.dx*grid.dy
			end
		end

	end
	return output
end




function PeskinDelta(X::StepRangeLen{Float64, Float64, Float64, Int64})
	delta = zeros(4)
	delta[1] = (5.0 + 2.0*X[1] - sqrt(-7. - 12.0*X[1] - 4.0*X[1]^2) )/8.0
	delta[2] = (3.0 + 2.0*X[2] + sqrt(1. - 4.0*X[2] - 4.0*X[2]^2) )/8.0
	delta[3] = (3.0 - 2.0*X[3] + sqrt(1. + 4.0*X[3] - 4.0*X[3]^2) )/8.0
	delta[4] = (5.0 - 2.0*X[4] - sqrt(-7. + 12.0*X[4] - 4.0*X[4]^2) )/8.0
	return delta
end

function  BndIntegral(data::Vector{Float64},bnd::PeriodicLagrangianBoundary)
	if ~(length(data) == bnd.N)
		println("Size is not same.")
	end

	summand = @. data*bnd.ds0;
	output = sum(summand);
	return output
end

BndIntegral (generic function with 1 method)